# V2

In [ ]:
system_i/tdc_2ch_0/U0/tdc1/b1_gen[0].b1_buffer_inst

In [2]:
OUTPUT_FILE = "tdc_placement.xdc"

IDX_TDC_2CH = 0
COMMON_PATH = f"system_i/tdc_2ch_{IDX_TDC_2CH}/U0"

SLICE_X0=36
SLICE_Y0=51

# ----------------------------------------------------------------------------------------------------------------------------------------------------------------
# generate a list of names for input LUT tree and sampling registers
def eval_f(f_string, local_dict):
    return eval(f'f"""{f_string}"""', globals() ,local_dict)
    
b0 = "b0_gen/lut1_i"
b1 = "b1_gen[{ib1}].b1_buffer_inst/lut1_i"
b2 = "b2_gen[{ib2}].b2_buffer_inst/lut1_i"
b3 = "b3_gen[{ib3}].b3_buffer_inst/lut1_i"
list_Bluts =[eval_f(b3 ,{"ib3":0}),
             eval_f(b2 ,{"ib2":0}),
             eval_f(b3 ,{"ib3":1}),
             eval_f(b1 ,{"ib1":0}),
             eval_f(b3 ,{"ib3":2}),
             eval_f(b2 ,{"ib2":1}),
             eval_f(b3 ,{"ib3":3}),
             eval_f(b0 ,{"ib0":0}),
             eval_f(b3 ,{"ib3":4}),
             eval_f(b2 ,{"ib2":2}),
             eval_f(b3 ,{"ib3":5}),
             eval_f(b1 ,{"ib1":1}),
             eval_f(b3 ,{"ib3":6}),
             eval_f(b2 ,{"ib2":3}),
             eval_f(b3 ,{"ib3":7}),]

Sregs = "sampler_gen[{iS1}].fine_samples_S_reg[{iS2}]"
list_Sregs = [eval(f'f"""{Sregs}"""', globals() ,{"iS1":i, "iS2":i}) for i in range(8)] + [eval(f'f"""{Sregs}"""', globals() ,{"iS1":i, "iS2":i+8}) for i in range(8)]

In [3]:
constraint_str = f"""
## False path
# hit signal
set_false_path -from [get_cells -hierarchical *hit_signal*]
set_false_path -from [get_cells -hierarchical *hit_signal_extended_reg*]
# reset signal
set_false_path -from [get_ports {COMMON_PATH}/tdc1/rst_n]
set_false_path -from [get_ports {COMMON_PATH}/tdc2/rst_n]
#  input signal
set_false_path -from [get_pins -hierarchical *hit_detected_edge_reg/C] -to [get_pins -hierarchical *fine_samples_Q_reg*/CLR]

## TDC1
# Bus skew constraint for the LUT buffer tree
set_bus_skew 0.1 -from [get_pins -hierarchical -filter {{NAME =~ "{COMMON_PATH}/tdc1/hit_signal_extended_reg/C"}}] -to [get_pins -hierarchical -filter {{NAME =~ "{COMMON_PATH}/tdc1/*fine_samples_S_reg[*]/D"}}]

# Bus skew for Alignment of 16 phases after the sampler
set_bus_skew -from [get_pins -hierarchical -filter {{NAME =~ "{COMMON_PATH}/tdc1/*fine_samples_S_reg[*]/C"}}] -to [get_pins -hierarchical -filter {{NAME =~ "{COMMON_PATH}/tdc1/*fine_samples_Q_reg[*]/D"}}] 0.150
# Together with max delay 
set_max_delay -from [get_cells -hierarchical -filter {{NAME =~ "{COMMON_PATH}/tdc1/*fine_samples_S_reg[*]"}}] -to [get_cells -hierarchical -filter {{NAME =~ "{COMMON_PATH}/tdc1/*fine_samples_Q_reg[*]"}}] 2 -datapath_only
set_max_delay -from [get_cells -hierarchical -filter {{NAME =~ "{COMMON_PATH}/tdc1/*fine_samples_Q_reg[*]"}}] -to [get_cells -hierarchical -filter {{NAME =~ "{COMMON_PATH}/tdc1/*fine_samples_C_reg[*]"}}] 2 -datapath_only
set_max_delay -from [get_cells -hierarchical -filter {{NAME =~ "{COMMON_PATH}/tdc1/*fine_samples_C_reg[*]"}}] -to [get_cells -hierarchical -filter {{NAME =~ "{COMMON_PATH}/tdc1/*latched_fine_time_reg[*]"}}] 1

## TDC2
# Bus skew constraint for the LUT buffer tree
set_bus_skew 0.15 -from [get_pins -hierarchical -filter {{NAME =~ "{COMMON_PATH}/tdc2/hit_signal_extended_reg/C"}}] -to [get_pins -hierarchical -filter {{NAME =~ "{COMMON_PATH}/tdc2/*fine_samples_Q_reg[*]/D"}}]

# Bus skew for Alignment of 16 phases after the sampler
set_bus_skew -from [get_pins -hierarchical -filter {{NAME =~ "{COMMON_PATH}/tdc2/*fine_samples_S_reg[*]/C"}}] -to [get_pins -hierarchical -filter {{NAME =~ "{COMMON_PATH}/tdc2/*fine_samples_Q_reg[*]/D"}}] 0.150
# Together with max delay 
set_max_delay -from [get_cells -hierarchical -filter {{NAME =~ "{COMMON_PATH}/tdc2/*fine_samples_S_reg[*]"}}] -to [get_cells -hierarchical -filter {{NAME =~ "{COMMON_PATH}/tdc2/*fine_samples_Q_reg[*]"}}] 2 -datapath_only
set_max_delay -from [get_cells -hierarchical -filter {{NAME =~ "{COMMON_PATH}/tdc2/*fine_samples_Q_reg[*]"}}] -to [get_cells -hierarchical -filter {{NAME =~ "{COMMON_PATH}/tdc2/*fine_samples_C_reg[*]"}}] 2 -datapath_only
set_max_delay -from [get_cells -hierarchical -filter {{NAME =~ "{COMMON_PATH}/tdc2/*fine_samples_C_reg[*]"}}] -to [get_cells -hierarchical -filter {{NAME =~ "{COMMON_PATH}/tdc2/*latched_fine_time_reg[*]"}}] 1


## Place the remaining logic in the nearby Pblock
create_pblock Pblock_TDC{IDX_TDC_2CH}
set_property IS_SOFT FALSE [get_pblocks Pblock_TDC{IDX_TDC_2CH}]
resize_pblock [get_pblocks Pblock_TDC{IDX_TDC_2CH}] -add {{SLICE_X{SLICE_X0-2}Y{SLICE_Y0}:SLICE_X{SLICE_X0+7}Y{SLICE_Y0+14}}}
add_cells_to_pblock [get_pblocks Pblock_TDC{IDX_TDC_2CH}] [get_cells -quiet -hierarchical -filter {{NAME =~ "{COMMON_PATH}/tdc1/*"}}] 
add_cells_to_pblock [get_pblocks Pblock_TDC{IDX_TDC_2CH}] [get_cells -quiet -hierarchical -filter {{NAME =~ "{COMMON_PATH}/tdc2/*"}}] 


create_pblock Pblock_TDC{IDX_TDC_2CH}_out
set_property IS_SOFT FALSE [get_pblocks Pblock_TDC{IDX_TDC_2CH}_out]
resize_pblock [get_pblocks Pblock_TDC{IDX_TDC_2CH}_out] -add {{SLICE_X{SLICE_X0-8}Y{SLICE_Y0}:SLICE_X{SLICE_X0-3}Y{SLICE_Y0+14}}}
add_cells_to_pblock [get_pblocks Pblock_TDC{IDX_TDC_2CH}_out] [get_cells -quiet -hierarchical -filter {{NAME =~ "{COMMON_PATH}/fast_data_builder_inst/*"}}] 
add_cells_to_pblock [get_pblocks Pblock_TDC{IDX_TDC_2CH}_out] [get_cells -quiet -hierarchical -filter {{NAME =~ "{COMMON_PATH}/trigger/*"}}] 

"""
# print(constraint_str)

In [4]:
output_xdc = []
x_gap = 4

for iTDC in [1,2]:
    header_i = eval_f(COMMON_PATH+"/tdc{iTDC}/", {"iTDC":iTDC})
    
    # Bx LUT buffers
    for i in range(len(list_Bluts)):
        output_xdc.append(f"set_property DONT_TOUCH TRUE [get_cells {header_i+list_Bluts[i]}]") # First, ensure the cell is not optimized away
        output_xdc.append(f"set_property LOC SLICE_X{SLICE_X0+2+x_gap*(iTDC-1)}Y{SLICE_Y0+i} [get_cells {header_i+list_Bluts[i]}]") # Then, apply your LOC constraint
    
    
    # [S]ampling regs 
    for i in range(8):
        
        output_xdc.append(f"set_property DONT_TOUCH TRUE [get_cells {header_i+list_Sregs[i]}]")
        output_xdc.append(f"set_property LOC SLICE_X{SLICE_X0+x_gap*(iTDC-1)}Y{SLICE_Y0+i*2} [get_cells {header_i+list_Sregs[i]}]")
        output_xdc.append(f"set_property DONT_TOUCH TRUE [get_cells {header_i+list_Sregs[i+8]}]")
        output_xdc.append(f"set_property LOC SLICE_X{SLICE_X0+1+x_gap*(iTDC-1)}Y{SLICE_Y0+i*2} [get_cells {header_i+list_Sregs[i+8]}]")

    output_xdc.append("\n")

for line in output_xdc:
    constraint_str += line+"\n"

In [5]:
with open(OUTPUT_FILE, "w") as file:
    file.write(constraint_str)

In [19]:
print(constraint_str)


## False path
# hit signal
set_false_path -from [get_cells -hierarchical *hit_signal*]
set_false_path -from [get_cells -hierarchical *hit_signal_extended_reg*]
# reset signal
set_false_path -from [get_ports fifo_readout_i/tdc_2ch_0/U0/tdc1/rst_n]
set_false_path -from [get_ports fifo_readout_i/tdc_2ch_0/U0/tdc2/rst_n]
#  input signal
set_false_path -from [get_pins -hierarchical *hit_detected_edge_reg/C] -to [get_pins -hierarchical *fine_samples_Q_reg*/CLR]

## TDC1
# Bus skew constraint for the LUT buffer tree
set_bus_skew 0.1 -from [get_pins -hierarchical -filter {NAME =~ "fifo_readout_i/tdc_2ch_0/U0/tdc1/hit_signal_extended_reg/C"}] -to [get_pins -hierarchical -filter {NAME =~ "fifo_readout_i/tdc_2ch_0/U0/tdc1/*fine_samples_S_reg[*]/D"}]

# Bus skew for Alignment of 16 phases after the sampler
set_bus_skew -from [get_pins -hierarchical -filter {NAME =~ "fifo_readout_i/tdc_2ch_0/U0/tdc1/*fine_samples_S_reg[*]/C"}] -to [get_pins -hierarchical -filter {NAME =~ "fifo_readout_i/tdc_2ch_0/